# Video Sampling Example

This notebook demonstrates how to use the sampling functions to generate videos from text prompts.

In [ ]:
import sys
from sample import sample_videos, sample_single_video, prompts
from release_server import load_merge_config, load_all

In [ ]:
import torch
import gc
config = load_merge_config("configs/self_forcing_dmd_14b_server.yaml")
models = load_all(config)
print("✅ Models loaded and ready to use!")
gc.collect()
torch.cuda.empty_cache()


In [ ]:
from sample import GenerateParams
params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=9,
    seed=1234,
    kv_cache_num_frames=3,
    thresh_kv_scale = 1.,
    do_kv_recomp=True,
    num_denoising_steps=8,
)
prompts = [
    "A sorcerer stands with one hand outstretched, holding a roiling flame that coils and twists restlessly around his palm and fingers, sparks shooting off in unpredictable arcs. The fire spirals and lashes outward, wrapping around his arm like a living serpent before snapping back toward his hand in a continuous, fluid motion. His cloak ripples in the heat’s updraft as the flame flares and contracts, creating bursts of glowing embers that rise and scatter through the air",
]
results = sample.sample_videos(
    prompts_list=[p],
    models=models,  # Reuse loaded models
    params=params,
    output_dir="out_release",
    save_videos=True,
    fps=16
)

## Video to Video

In [ ]:

params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=9,
    seed=1234,
    kv_cache_num_frames=3,
    thresh_kv_scale = 1.,
    do_kv_recomp=True,
    num_denoising_steps=8,

    input_vid = "......" # ADD this
)
prompts = [
    "A massive wooden ship sails through a dark, stormy sea, its masts swaying violently as enormous waves crash against the hull. The ocean roars with relentless power, sending bursts of white spray over the deck as the crew struggles to keep control. Rain pours down in thick sheets, blurring the horizon into a gray haze. The camera moves with the ship’s motion, capturing the rhythm of the storm and the sheer force of the sea.",
]
results = sample.sample_videos(
    prompts_list=[p],
    models=models,  # Reuse loaded models
    params=params,
    output_dir="out_release",
    save_videos=True,
    fps=16
)